In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
dev_df=pd.read_csv('./output/dev_statement_section.csv')
dev_df

,statement,section,label
0,there is a 13.2% difference between the result...,"[""Outcome Measurement:"", ""Event-free Survival""...",Contradiction
1,Patients with significantly elevated ejection ...,"[""Inclusion criteria:"", ""Inclusion Criteria:"",...",Contradiction
2,a significant number of the participants in th...,"[""Adverse Events 1:"", ""Total: 20/167 (11.98%)""...",Contradiction
3,the primary trial does not report the PFS or o...,"[""Outcome Measurement:"", ""Local Control Using ...",Entailment
4,Prior treatment with fulvestrant or with a pho...,"[""Inclusion Criteria:"", ""Postmenopausal women ...",Contradiction
...,...,...,...
195,The the primary trial intervention involves on...,"[""INTERVENTION 1:"", ""Letrozole"", ""Participants...",Contradiction
196,the secondary trial reported 1 single case of ...,"[""Adverse Events 1:"", ""Total: 16/48 (33.33%)"",...",Entailment
197,the secondary trial and the primary trial do n...,"[""Outcome Measurement:"", ""Number of Patients W...",Entailment
198,the outcome measurement of the primary trial i...,"[""Outcome Measurement:"", ""Progression-free Sur...",Entailment


In [3]:
hypothesis_lst=dev_df['statement'].values.tolist()
len(hypothesis_lst)

200

In [4]:
evidence_lst=dev_df['section'].apply(lambda l:' '.join(json.loads(l))).values.tolist()
len(evidence_lst)

200

In [5]:
label2id={"Contradiction":0,"Entailment":1}
label_lst=dev_df['label'].apply(lambda x:label2id[x]).values.tolist()
len(label_lst)

200

In [6]:
import json
with open('output/all_evidences.json','r') as f:
    all_evidences = json.load(f)
evidence2id=dict([(all_evidences[i],i) for i in range(len(all_evidences))])

from nltk.tokenize import word_tokenize
tokenized_corpus = [word_tokenize(doc) for doc in all_evidences]

from rank_bm25 import BM25Okapi
bm25 = BM25Okapi(tokenized_corpus)

In [7]:
import numpy as np
scores=[]
for hypothesis,evidences in dev_df[['statement','section']].values:
    doc_scores = bm25.get_scores(word_tokenize(hypothesis))
    doc_scores = (doc_scores-np.min(doc_scores))/(np.max(doc_scores)-np.min(doc_scores))
    scores.append(np.mean([doc_scores[evidence2id[evi]] for evi in json.loads(evidences)]))
# scores

In [8]:
min(scores), max(scores)

(0.0, 0.6763911682156237)

In [9]:
from sklearn.metrics import average_precision_score,f1_score,precision_score,recall_score,accuracy_score

y_true=label_lst
results=[]
for thre in np.arange(0.0, 1.0, 0.1):
    y_prob=scores
    y_pred=[1 if a>thre else 0 for a in y_prob]
    results.append([
        thre,
        average_precision_score(y_true,y_prob),
        f1_score(y_true,y_pred),
        precision_score(y_true,y_pred),
        recall_score(y_true,y_pred),
        accuracy_score(y_true,y_pred)
    ])

import pandas as pd

pd.DataFrame(results,columns=['threshold','AVG_PREC','F1','PREC','REC','ACC'])

/home/ai/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,threshold,AVG_PREC,F1,PREC,REC,ACC
0,0.0,0.485564,0.643110,0.497268,0.91,0.495
1,0.1,0.485564,0.492308,0.505263,0.48,0.505
2,0.2,0.485564,0.144000,0.360000,0.09,0.465
3,0.3,0.485564,0.088496,0.384615,0.05,0.485
4,0.4,0.485564,0.055556,0.375000,0.03,0.490
5,0.5,0.485564,0.019417,0.333333,0.01,0.495
6,0.6,0.485564,0.019608,0.500000,0.01,0.500
7,0.7,0.485564,0.000000,0.000000,0.00,0.500
8,0.8,0.485564,0.000000,0.000000,0.00,0.500
9,0.9,0.485564,0.000000,0.000000,0.00,0.500
